In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install torchmetrics
!pip install pytorch_lightning
import torchmetrics
import pytorch_lightning as pl
from torch import optim
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP_HW2/')
#wiki_dataset = __import__('/content/drive/MyDrive/Colab Notebooks/NLP_HW2/dataset/wiki_dataset')
from dataset import wiki_dataset
from dataloader import wiki_dataloader
import pytorch_lightning.loggers as pl_loggers
import nltk
nltk.download('punkt')

     |████████████████████████████████| 397 kB 4.1 MB/s 
     |████████████████████████████████| 527 kB 4.2 MB/s 
     |████████████████████████████████| 952 kB 45.1 MB/s 
     |████████████████████████████████| 829 kB 41.6 MB/s 
     |████████████████████████████████| 133 kB 52.3 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 271 kB 53.6 MB/s 
     |████████████████████████████████| 144 kB 53.6 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=64640f3993b7b9c5219e22e838aab7d53f5aafc90dfe2d8a7268f4172e345fc6
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfull

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
class LSTM1(pl.LightningModule):
    def __init__(self, n_vocab,
                 embedding_size,
                 hidden_size,
                 num_layers,
                 seq_size):
        super(LSTM1, self).__init__()
        self.seq_size = seq_size
        self.embedding_size=embedding_size
        self.hidden_size=hidden_size
        self.lstm = nn.LSTM(input_size = embedding_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first=True, dropout=0.5)
        # nn.utils.clip_grad_norm_(self.lstm.parameters(), clip) - between backwards and optimizer step
        self.prev_state = None
        self.embed = nn.Embedding(n_vocab, embedding_size)
        self.loss = nn.CrossEntropyLoss()
        self.fc = nn.Linear(embedding_size, n_vocab) #transpose of embedding layer; need same weights but transposed

    def forward(self, x, prev_state):
        x = self.embed(x)
        #x = torch.flatten(x, start_dim=1)
        if self.prev_state is None:
            x , state = self.lstm(x)
        else:
            x, state = self.lstm(x, prev_state)
        x = x[:, -1, :]
        logits = self.fc(x) #logit from running x through linear layer

        return logits, state

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        data, label = batch
        logits, self.prev_state = self.forward(data, self.prev_state)
        self.prev_state = [state.detach() for state in self.prev_state] #holding onto the numbers, not the gradient
        #need to move tensors back to gpu/cuda
        print(f'{logits.shape}, {label.shape}')
        # l2_norm = sum(p.pow(2.0).sum() for p in self.parameters()).item()
        # l1_norm = sum(p.abs().sum() for p in model.parameters()).item()
        loss = self.loss(logits, label)  # + l2_norm + l1_norm
        tensorboard_logs = {'loss': {'train': loss.detach()}}
        self.log("training loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, label = batch
        logits, self.prev_state = self.forward(data, self.prev_state)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'val': loss.detach()}}
        self.log("validation loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        data, label = batch
        logits, self.prev_state = self.forward(data, self.prev_state)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'test': loss.detach()}}
        self.log("test loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}



In [3]:

# Load datasets
train = wiki_dataset('./wiki.train.txt', training=True, token_map='create', window=30)
valid = wiki_dataset('./wiki.valid.txt', training=False, token_map=train.token_map, window=30)
test = wiki_dataset('./wiki.test.txt', training=False, token_map=train.token_map, window=30)
datasets = [train, valid, test]

In [4]:
# Load dataloader
dataloader = wiki_dataloader(datasets=datasets, batch_size=20)

In [5]:
# Make model and train
model = LSTM1(n_vocab=len(train.unique_tokens),
              num_layers=2,
              seq_size=30,
              embedding_size=100,
              hidden_size=100)

In [10]:
tb_logger = pl_loggers.TensorBoardLogger("./lightning_logs/", name="ff")
trainer = pl.Trainer(gradient_clip_val=0.5, logger=tb_logger, max_epochs=10, gpus=1)
trainer.fit(model, dataloader)
result = trainer.test(model, dataloader)
print(result)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | lstm  | LSTM             | 161 K 
1 | embed | Embedding        | 2.9 M 
2 | loss  | CrossEntropyLoss | 0     
3 | fc    | Linear           | 2.9 M 
-------------------------------------------
6.0 M     Trainable params
0         Non-trainable params
6.0 M     Total params
23.884    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: ignored